In [28]:
import pandas as pd
import numpy  as np
from sklearn.model_selection import train_test_split
from skrebate import ReliefF
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [29]:
df = pd.read_csv('data.csv')

In [30]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [32]:
df.drop('id',axis=1,inplace=True)

In [33]:
df.drop('Unnamed: 32',axis=1,inplace=True)

In [34]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['encoded_diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

In [35]:
df.drop('diagnosis',axis=1,inplace=True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              569 non-null    float64
 1   texture_mean             569 non-null    float64
 2   perimeter_mean           569 non-null    float64
 3   area_mean                569 non-null    float64
 4   smoothness_mean          569 non-null    float64
 5   compactness_mean         569 non-null    float64
 6   concavity_mean           569 non-null    float64
 7   concave points_mean      569 non-null    float64
 8   symmetry_mean            569 non-null    float64
 9   fractal_dimension_mean   569 non-null    float64
 10  radius_se                569 non-null    float64
 11  texture_se               569 non-null    float64
 12  perimeter_se             569 non-null    float64
 13  area_se                  569 non-null    float64
 14  smoothness_se            5

In [37]:
X = df.drop('encoded_diagnosis', axis=1).values
y = df['encoded_diagnosis'].values

In [38]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
# Feature selection using ReliefF algorithm
reliefF = ReliefF(n_neighbors=10, n_features_to_select=10)
X_train_relief = reliefF.fit_transform(X_train, y_train)
X_test_relief = reliefF.transform(X_test)

In [44]:
# Define SVM classifiers
svm_linear = SVC(kernel='linear')
svm_rbf = SVC(kernel='rbf')

# Create pipelines for SVM with PCA
svm_linear_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),  # Adjust the number of components as needed
    ('svm_linear', svm_linear)
])

svm_rbf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),  # Adjust the number of components as needed
    ('svm_rbf', svm_rbf)
])

# Fit the models
svm_linear_pipeline.fit(X_train_relief, y_train)
svm_rbf_pipeline.fit(X_train_relief, y_train)

# Evaluate the models
linear_accuracy = svm_linear_pipeline.score(X_test_relief, y_test)
rbf_accuracy = svm_rbf_pipeline.score(X_test_relief, y_test)

print("Accuracy of SVM (linear kernel): {:.2f}%".format(linear_accuracy * 100))
print("Accuracy of SVM (RBF kernel): {:.2f}%".format(rbf_accuracy * 100))

Accuracy of SVM (linear kernel): 97.37%
Accuracy of SVM (RBF kernel): 97.37%


In [40]:
pca = PCA(n_components=5)  # Adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train_relief)
X_test_pca = pca.transform(X_test_relief)

In [45]:
# Define SVM classifiers
svm_linear = SVC(kernel='linear')
svm_rbf = SVC(kernel='rbf')

# Create pipelines for SVM with PCA
svm_linear_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),  # Adjust the number of components as needed
    ('svm_linear', svm_linear)
])

svm_rbf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),  # Adjust the number of components as needed
    ('svm_rbf', svm_rbf)
])

# Fit the models
svm_linear_pipeline.fit(X_train_pca, y_train)
svm_rbf_pipeline.fit(X_train_pca, y_train)

# Evaluate the models
linear_accuracy = svm_linear_pipeline.score(X_test_pca, y_test)
rbf_accuracy = svm_rbf_pipeline.score(X_test_pca, y_test)

print("Accuracy of SVM (linear kernel): {:.2f}%".format(linear_accuracy * 100))
print("Accuracy of SVM (RBF kernel): {:.2f}%".format(rbf_accuracy * 100))

Accuracy of SVM (linear kernel): 95.61%
Accuracy of SVM (RBF kernel): 97.37%


In [46]:
# Assuming X_train_relief is a numpy array
num_features = X_train_pca.shape[1]
print("Number of features:", num_features)

Number of features: 5


In [47]:
# Assuming X_train_pca is a numpy array
num_features = X_train_pca.shape[1]

# Print the indices of the important features
print("Indices of important features:")
for index in range(num_features):
    print(index)


Indices of important features:
0
1
2
3
4


In [48]:
# Assuming X_train_relief is a numpy array
num_features = X_train_relief.shape[1]
print("Number of features:", num_features)

Number of features: 10


In [49]:
# Assuming X_train_pca is a numpy array
num_features = X_train_relief.shape[1]

# Print the indices of the important features
print("Indices of important features:")
for index in range(num_features):
    print(index)

Indices of important features:
0
1
2
3
4
5
6
7
8
9
